In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from scipy import stats
from copulae import StudentCopula

# Data acquisition
tickers = ['AAPL', 'MSFT', 'GOOGL']
data = yf.download(tickers, start="2019-01-01", end="2024-01-01")

# Pricing parameters
risk_free_rate = 0.04  # 4% annual rate
T = 1.0                # 1-year horizon

if 'Adj Close' in data.columns:
    adj_close = data['Adj Close']
else:
    adj_close = data['Close']

returns = np.log(adj_close / adj_close.shift(1)).dropna()

# Volatility filtering (GARCH)
std_residuals_dict = {}
current_annual_vol = {}

for ticker in tickers:
    # Use dist='t' in GARCH to match marginal distribution logic
    model = arch_model(returns[ticker] * 100, vol='Garch', p=1, q=1, dist='t')
    res = model.fit(disp='off')
    
    # Standardized Residuals
    std_residuals_dict[ticker] = res.resid / res.conditional_volatility
    
    # Extract the latest conditional volatility and annualize it
    # This represents the "current" market state for our projection
    current_annual_vol[ticker] = (res.conditional_volatility.iloc[-1] * np.sqrt(252)) / 100

cleaned_data = pd.DataFrame(std_residuals_dict)

#Marginal distributions
dist_params = {}
uniform_data = pd.DataFrame()

for ticker in tickers:
    params = stats.t.fit(cleaned_data[ticker])
    dist_params[ticker] = params
    uniform_data[ticker] = stats.t.cdf(cleaned_data[ticker], *params)

#Copula calibration
cop = StudentCopula(dim=len(tickers))
cop.fit(uniform_data)

# Monte Carlo simulations (risk-neutral)
num_sim = 100000
sim_uniform = cop.random(num_sim).values 
sim_prices = pd.DataFrame()
current_prices = adj_close.iloc[-1]

for i, ticker in enumerate(tickers):
    df, loc, scale = dist_params[ticker]
    # correlated shocks from the Copula
    shocks = stats.t.ppf(sim_uniform[:, i], df, loc, scale)
    
    # RISK-NEUTRAL GEOMETRIC BROWNIAN MOTION FORMULA:
    # S_T = S_0 * exp((r - 0.5 * sigma^2) * T + sigma * sqrt(T) * Z)
    sigma = current_annual_vol[ticker]
    drift = (risk_free_rate - 0.5 * (sigma**2)) * T
    diffusion = sigma * np.sqrt(T) * shocks
    
    sim_prices[ticker] = current_prices[ticker] * np.exp(drift + diffusion)

#Pricing of the Worst-Of option
sim_performance = sim_prices / current_prices
worst_of_perf = sim_performance.min(axis=1)

strike = 0.90
# This is a 'Put' on the worst performer (protection against drops)
payoffs = np.maximum(0, strike - worst_of_perf)

#Fair Price = Expected Payoff * Discount Factor (e^-rT)
price_estimate = payoffs.mean() * np.exp(-risk_free_rate * T)

print(f"Results (Risk-Neutral Pricing):")
print(f"Underlying Assets: {tickers}")
print(f"Risk-Free Rate: {risk_free_rate:.1%}")
print(f"Protection Level (Strike): {strike:.0%}")
print(f"Fair Option Price: {price_estimate:.4%}")
standard_error = payoffs.std() / np.sqrt(num_sim)
print(f"95% Confidence Interval: [{price_estimate - 1.96*standard_error:.4%}, {price_estimate + 1.96*standard_error:.4%}]")

[*********************100%***********************]  3 of 3 completed


              AAPL         MSFT        GOOGL
count  1257.000000  1257.000000  1257.000000
mean      0.500000     0.500000     0.500000
std       0.288560     0.288560     0.288560
min       0.000795     0.000795     0.000795
25%       0.250397     0.250397     0.250397
50%       0.500000     0.500000     0.500000
75%       0.749603     0.749603     0.749603
max       0.999205     0.999205     0.999205
Results (Risk-Neutral Pricing):
Underlying Assets: ['AAPL', 'MSFT', 'GOOGL']
Risk-Free Rate: 4.0%
Protection Level (Strike): 90%
Fair Option Price: 4.0273%
95% Confidence Interval: [3.9762%, 4.0785%]
